In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.insert(1, 'src/')
import utilities
import config
import csv

from fastai.text import *

In [3]:
# Finetune DATA

SARC_DATA = config.SARC_DATA
SARC_MAIN = config.SARC_MAIN
SARC_POL = config.SARC_POL

train_path = SARC_MAIN + 'train-unbalanced.csv'
test_path = SARC_MAIN + 'test-unbalanced.csv'
comment_path = SARC_MAIN + 'comments.json'

train_docs, test_docs, train_labels, test_labels = \
    utilities.load_sarc_responses(train_path, test_path, comment_path)

In [5]:
learn_data = []

# I keep getting a databunch load error if I don't add a ' ' (or some other non empty string) to the end.

for index, sample in enumerate(train_labels):
    ancestor = train_docs['ancestors'][index][0]
    learn_data.append([-1, ancestor])
    for i, label in enumerate(sample):
        response = train_docs['responses'][index][i]
        learn_data.append([int(label), response + ' '])

for index, sample in enumerate(test_labels):
    ancestor = test_docs['ancestors'][index][0]
    learn_data.append([-1, ancestor])
    for i, label in enumerate(sample):
        response = test_docs['responses'][index][i]
        learn_data.append([int(label), response + ' '])

In [6]:
with open('finetune_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(learn_data)


In [7]:
# Load databunch for lm training
ml_bunch = TextLMDataBunch.from_csv(path='.', csv_name='finetune_data.csv')

# Save databunch
ml_bunch.save('lm_databunch')

/home/gaetano/anaconda3/envs/sarc/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [2]:
# Classifier DATA

SARC_DATA = config.SARC_DATA
SARC_MAIN = config.SARC_MAIN
SARC_POL = config.SARC_POL

train_path = SARC_MAIN + 'train-balanced.csv'
test_path = SARC_MAIN + 'test-balanced.csv'
comment_path = SARC_MAIN + 'comments.json'

train_docs, test_docs, train_labels, test_labels = \
    utilities.load_sarc_responses(train_path, test_path, comment_path)

In [3]:
learn_data = []

for index, sample in enumerate(train_labels):
    ancestor = train_docs['ancestors'][index][0]
    for i, label in enumerate(sample):
        response = train_docs['responses'][index][i]
        learn_data.append([int(label), ancestor + ' ^ ' + response])

In [4]:
with open('class_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(learn_data)

In [5]:
# load databunch for classification training
class_bunch = TextClasDataBunch.from_csv(path='.', csv_name='class_data.csv')

# save databunch
class_bunch.save('class_databunch')

/home/gaetano/anaconda3/envs/sarc/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
